In [3]:
## Import Libraries

import pandas as pd
import numpy as np
import string
import re
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.manifold import TSNE    

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from imblearn.over_sampling import SMOTE


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
# load data
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)

In [5]:
# remove rows with missing values
def clean_df(data):
    df = data.dropna().drop_duplicates() 
    # drop rows where medical specialty appears less than 100 times
    df = df[df.groupby("medical_specialty")["medical_specialty"].transform('size') > 100]
    print("Number of rows after removing medical specialties with less than 50 samples:", len(df.index))
    # remove unnecessary columns, only keep transcriptions and medical_specialty columns
    return df[['transcription', 'medical_specialty']]


# remove punctuation and lowercase and lemmatizer
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def remove_punct_lower(data):
    '''remove punctuation and lowercase'''
    data["transcription_c"] = data["transcription"].apply(lambda x: x.lower().translate(str.maketrans('','', string.punctuation)))
    return data

def lemmatize_words(data):
    '''lemmatize words, remove stopwords'''
    data["transcription_c"] = data["transcription_c"].apply(lambda x: [lemmatizer.lemmatize(x) for x in word_tokenize(x) if x not in (stop)])
    return data
 


#apply on dataset
df_m = clean_df(df)
df_test = remove_punct_lower(df_m)
df_test = lemmatize_words(df_test)


df_test.head(2)

Number of rows after removing medical specialties with less than 50 samples: 2976


,transcription,medical_specialty,transcription_c
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"[2d, mmode, 1, left, atrial, enlargement, left..."
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"[1, left, ventricular, cavity, size, wall, thi..."


In [1]:
!pip3 install ../data/en_ner_bionlp13cg_md-0.5.1

Processing /Users/hannahpetry/Desktop/Work Project/NLP_Masterthesis/data/en_ner_bionlp13cg_md-0.5.1
  Preparing metadata (setup.py) ... done
  Created wheel for en-ner-bionlp13cg-md: filename=en_ner_bionlp13cg_md-0.5.1-py3-none-any.whl size=120241147 sha256=a129e3af6632c973998a4e4c1ed6847eefa021aa97916119d25140e0c1a8d344
  Stored in directory: /Users/hannahpetry/Library/Caches/pip/wheels/ba/dc/7b/d8d45322e45cc36e3226aef67695c2eaa65d737e0def574ef2
Successfully built en-ner-bionlp13cg-md


In [6]:
# NLP with Spacy
import spacy
import en_ner_bionlp13cg_md
nlp = en_ner_bionlp13cg_md.load()
def medical_entities( text):
    entities = []
    doc = nlp(text)
    for ent in doc.ents:
        entities.append(ent.text)
    return set(entities)#' ,'.join(entities)


df_test['transcription_c'] = [','.join(map(str, l)) for l in df_test['transcription_c']]
df_test['transcription_f'] = df_test['transcription_c'].apply(medical_entities)
df_test.head()

,transcription,medical_specialty,transcription_c,transcription_f
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"2d,mmode,1,left,atrial,enlargement,left,atrial...","{valve, left, pulmonary, ventricular, mitral}"
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"1,left,ventricular,cavity,size,wall,thickness,...","{leaflet, ventricle, valve, left, atrium, lipo..."
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary,"2d,echocardiogrammultiple,view,heart,great,ves...","{aorta, valve, vessel, left, atrium, pulmonary..."
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary,"description1,normal,cardiac,chamber,size2,norm...","{valve, left, ventricular, cardiac, mitral}"
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary,"2d,study1,mild,aortic,stenosis,widely,calcifie...","{ventricle, left, ventricular, heart, mitral}"


In [15]:
df_test.transcription_f = df_test.transcription_f.astype(str)
df_test.transcription_f

3       {'valve', 'left', 'pulmonary', 'ventricular', ...
4       {'leaflet', 'ventricle', 'valve', 'left', 'atr...
7       {'aorta', 'valve', 'vessel', 'left', 'atrium',...
9       {'valve', 'left', 'ventricular', 'cardiac', 'm...
11      {'ventricle', 'left', 'ventricular', 'heart', ...
                              ...                        
4967    {'circumflex', 'aorta', 'renal', 'pda', 'pci',...
4968    {'aspirin', 'vessel', 'myocardial', 'femoral',...
4971    {'blood', 'ear', 'valve', 'fat', 'muscle', 'an...
4972    {'adenosine', 'nuclear', 'myocardial', 'patien...
4975    {'circumflex', 'pci', 'lipid', 'gerd', 'left',...
Name: transcription_f, Length: 2976, dtype: object

In [16]:
# create sparse matrix from transcription_f
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_test.transcription_f)
X.shape

(2976, 2702)

In [20]:
# Create X and y and apply PCA to reduce dimensionality of features
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X.toarray())
labels = df_test['medical_specialty'].tolist()

In [18]:
# Create a class to select numerical or categorical columns 
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [22]:
# balance out dataset with SMOTE, creates synthetic samples of the minority classes
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_reduced, labels)

In [23]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

In [24]:
# build model with finetuned hyperparameters
clf = LogisticRegression(random_state=42, penalty= 'l1', solver= 'saga', multi_class='multinomial', C=1)
lr = Pipeline(steps=[('classifier', clf)]).fit(X_train, y_train)

/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [25]:
category_list = df_test.medical_specialty.unique()
# predict and evaluate
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred, target_names=category_list))

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary       0.71      0.75      0.73       208
                       Urology       0.71      0.55      0.62       218
              General Medicine       0.75      0.82      0.79       191
                       Surgery       0.77      0.83      0.80       213
 SOAP / Chart / Progress Notes       0.72      0.81      0.77       193
                     Radiology       0.76      0.89      0.82       202
                    Orthopedic       0.66      0.72      0.69       202
       Obstetrics / Gynecology       0.54      0.50      0.52       191
                     Neurology       0.68      0.82      0.74       225
              Gastroenterology       0.53      0.23      0.32       203
    Consult - History and Phy.       0.87      0.92      0.89       205

                      accuracy                           0.71      2251
                     macro avg       0.70      0.71      0.70 

In [26]:
# ROC curve
# Generate class membership probabilities
y_preb_probs = lr.predict_proba(X_test)
score = roc_auc_score(y_test, y_preb_probs, average="weighted", multi_class="ovr")
print('Average AUROC score of', round(score,4))

Average AUROC score of 0.9517


In [27]:
# version of accuracy that also integrates measurements of chance and class imbalance
# Generally, a score above 0.8 is considered excellent
cohen_kappa_score(y_test, y_pred)

0.6847131496502603

In [28]:
# Predict probabilities for a sample from test set
prob_array = lr.predict_proba(X_test)[0,:]
prob_df = pd.DataFrame(prob_array, index=category_list, columns=['Probability']).sort_values(by='Probability', ascending=False)
prob_df

,Probability
Radiology,0.684487
Gastroenterology,0.107089
Obstetrics / Gynecology,0.093361
Consult - History and Phy.,0.037043
General Medicine,0.020825
Urology,0.018470
Neurology,0.014036
Cardiovascular / Pulmonary,0.012928
Orthopedic,0.005452
SOAP / Chart / Progress Notes,0.003385
